<a href="https://colab.research.google.com/github/advik-7/Image_Encryption/blob/main/Discriminator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_discriminator():
    image_input = layers.Input(shape=(256, 256, 3))  # Input shape

    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(image_input)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(512, kernel_size=4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Flatten()(x)
    validity = layers.Dense(1, activation='sigmoid')(x)
    discriminator_model = Model(image_input, validity)

    def discriminator_loss(real, fake):
        real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(real), real)
        fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake), fake)
        return (real_loss + fake_loss) * 0.5

    optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    def train_step(real_images, fake_images):
        with tf.GradientTape() as tape:
            # Get the validity of real and fake images
            real_validity = discriminator_model(real_images, training=True)
            fake_validity = discriminator_model(fake_images, training=True)

            # = loss
            loss = discriminator_loss(real_validity, fake_validity)

        #Optimixer
        gradients = tape.gradient(loss, discriminator_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, discriminator_model.trainable_variables))

        return loss

    return discriminator_model, train_step


